In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm, trange
import os
import matplotlib.cm as cm
import seaborn as sns

In [2]:
CSV_PATH = './dataset/mswc21/csv/TRAIN.csv'
FEATURES_PATH = './dataset/mswc21/features/all_train_embeddings_1024'
# FEATURES_PATH = './dataset/mswc21/features/all_train_embeddings_1024'

In [3]:
df = pd.read_csv(CSV_PATH, delimiter=',').drop(columns=['Unnamed: 0'])
df = df[df['mode']=='train']
# df = df[df['mode'] == 'val']
# df = df[df['language'] == 'tt']
df['path'] = df['path'].apply(lambda x: x.replace('.wav', '.pt'))
df.head()

,mode,label,path,language,gender
0,train,этого,ru/clips/этого/common_voice_ru_18956507.pt,ru,m
1,train,резолюции,ru/clips/резолюции/common_voice_ru_19157550.pt,ru,m
2,train,менее,ru/clips/менее/common_voice_ru_19288549.pt,ru,m
3,train,kunnen,nl/clips/kunnen/common_voice_nl_22221094.pt,nl,m
5,train,слово,ru/clips/слово/common_voice_ru_18963836.pt,ru,m


In [4]:
def get_vector(matrix):
    return matrix.flatten()

def get_vectors(paths):
    vectors = []
    for path in tqdm(paths):
        matrix = torch.load(os.path.join(FEATURES_PATH, path), map_location=torch.device('cpu'))
        vectors.append(matrix)
#         vectors.append(get_vector(matrix))
    return np.stack(vectors)
    
vectors = get_vectors(df['path'].values)

100%|█████████████████████████████████| 549858/549858 [03:28<00:00, 2641.97it/s]


In [10]:
del initial_vectors

In [5]:
from sklearn.cluster import KMeans
import numpy as np
N = 100
kmeans = KMeans(n_clusters=N, random_state=29)

In [6]:
%%time
kmeans.fit(vectors)

CPU times: user 2h 48min 58s, sys: 3h 10min 32s, total: 5h 59min 30s
Wall time: 42min 30s


KMeans(n_clusters=100, random_state=29)

In [7]:
from sklearn.metrics import pairwise_distances_argmin_min


def generate_points(kmeans, vectors, exp_no):
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, vectors)
    randoms = []
    for i in trange(len(kmeans.cluster_centers_)):
        cluster_points = []
        for j, label in enumerate(kmeans.labels_):
            if label == i:
                cluster_points.append(j)
        
        randoms.append(np.random.choice(cluster_points, size=exp_no, replace=True))
    return closest, randoms

In [ ]:
closest, randoms = generate_points(kmeans, vectors, 10)

 86%|████████████████████████████████████      | 86/100 [01:07<00:10,  1.29it/s]

In [ ]:
def get_trains(df, closest, randoms):
    dfs = []
    for i in trange(len(randoms[0])):
        indices = list(closest) + [x[i] for x in randoms]
        dfs.append(df.iloc[indices, :])
    return dfs

In [ ]:
trains = get_trains(df, closest, randoms)

In [79]:
print(len(closest))
print(closest)

100
[501054 192241 155347 419361 540224 357041 396672 257284 152062 147448
 374738 298687 155657 361345 127074 321197 233836 413263  89985 273980
 195825 279788  20171 267481 197803 203753 188438  13262 106569 222173
 521303 359387 223776 222952 385420 371067  77657 315678 459905 466691
 333143 315957  57494 415540 538787 389978 376253 525069 117936 398183
 465542 230995 105463 117983 304551   1060  97254 544824 491445 469310
 372788 299655 212684 274463 523839 173853 253457 549140 197230 339805
 142264 106354 446478 162899  48461 356121 510200  83498 186260  80124
 309486 150244 222737 312178 452795 175426 388045 477607 435482 522334
 188772 535813 412568 532101  93392  95439 354401  29960 453350 138799]


In [82]:

print(df)# display(trains[0])
print(df.iloc[closest, :])

         mode      label                                            path  \
0       train      этого      ru/clips/этого/common_voice_ru_18956507.pt   
1       train  резолюции  ru/clips/резолюции/common_voice_ru_19157550.pt   
2       train      менее      ru/clips/менее/common_voice_ru_19288549.pt   
3       train     kunnen     nl/clips/kunnen/common_voice_nl_22221094.pt   
5       train      слово      ru/clips/слово/common_voice_ru_18963836.pt   
...       ...        ...                                             ...   
688517  train      aveva      it/clips/aveva/common_voice_it_19162642.pt   
688518  train       duen       eu/clips/duen/common_voice_eu_18688125.pt   
688519  train      ginen      eu/clips/ginen/common_voice_eu_18834236.pt   
688522  train      tutto      it/clips/tutto/common_voice_it_20272884.pt   
688523  train     икенче     tt/clips/икенче/common_voice_tt_17826470.pt   

       language gender  
0            ru      m  
1            ru      m  
2           

In [ ]:
import os

path = './dataset/mswc21/experiments/kmeans_all_train_1024'
os.makedirs(path, exist_ok=True)
for i, df_to_save in enumerate(trains):
    df_to_save.to_csv(os.path.join(path, f'{i}.csv'))

100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 2831.12it/s]


,mode,label,path,language,gender
167873,train,homens,pt/clips/homens/common_voice_pt_20470720.pt,pt,m
330940,train,drogę,pl/clips/drogę/common_voice_pl_20884723.pt,pl,m
94790,train,организации,ru/clips/организации/common_voice_ru_20716284.pt,ru,m
455578,train,essa,it/clips/essa/common_voice_it_19996981.pt,it,n
379246,train,principale,it/clips/principale/common_voice_it_18845764.pt,it,m
...,...,...,...,...,...
487712,train,konas,eo/clips/konas/common_voice_eo_17900367.pt,eo,m
95854,train,договора,ru/clips/договора/common_voice_ru_19116667.pt,ru,m
143465,train,развитие,ru/clips/развитие/common_voice_ru_19372750.pt,ru,m
422171,train,наши,ru/clips/наши/common_voice_ru_18970903.pt,ru,m


,mode,label,path,language,gender
167873,train,homens,pt/clips/homens/common_voice_pt_20470720.pt,pt,m
330940,train,drogę,pl/clips/drogę/common_voice_pl_20884723.pt,pl,m
94790,train,организации,ru/clips/организации/common_voice_ru_20716284.pt,ru,m
455578,train,essa,it/clips/essa/common_voice_it_19996981.pt,it,n
379246,train,principale,it/clips/principale/common_voice_it_18845764.pt,it,m
...,...,...,...,...,...
246706,train,организация,ru/clips/организация/common_voice_ru_20534869.pt,ru,m
563643,train,anos,pt/clips/anos/common_voice_pt_19422587.pt,pt,m
340099,train,więc,pl/clips/więc/common_voice_pl_20629267.pt,pl,m
385310,train,heeft,nl/clips/heeft/common_voice_nl_17701728.pt,nl,m


In [63]:
print(df.iloc[closest, :]['label'].sort_values())
print(len(df.iloc[closest, :]['label']))
print(len(df.iloc[closest, :]['label'].unique()))

159707            batez
300664          beharko
570613             bizi
282133           coisas
381441     czterdzieści
330940            drogę
594710           eniris
38307          eskatzen
455578             essa
128201           essere
428473            faris
350581          gertatu
195019            ginen
101247            havas
241389            havas
598914            hemen
542249        hizkuntza
167873           homens
605236          inglese
684606            każdy
139727         komencis
29859            kredas
184417             mimo
223771           muitos
262857             mãos
177844             oczy
597968       oczywiście
322382           okazas
316557           okazis
588517            poste
670915            primo
379246       principale
56521            questo
235806            senza
534179             seus
207744           spesso
495630            stato
419809             tiuj
369503            venas
602296            venis
671838            volas
72026           

In [47]:
print(initial_vectors[0].shape)

(1024,)


In [18]:
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [96]:
vectors_2d = PCA(n_components=2, svd_solver='full', random_state=29).fit_transform(kmeans.cluster_centers_)

In [97]:
print(vectors_2d[:2])

[[12.534813    0.49326304]
 [-3.6342702  -5.4910727 ]]


In [98]:
print(sorted([-x[0] for x in vectors_2d])[:10])
print(sorted([-x[1] for x in vectors_2d])[:10])

[-21.729357, -17.128513, -16.891571, -15.78923, -13.58545, -12.534813, -11.95319, -11.572049, -11.506225, -8.9423485]
[-16.77165, -14.76462, -12.449821, -12.369804, -9.3887205, -9.341587, -9.168316, -9.063241, -6.8312593, -5.4426403]


In [99]:
print(vectors_2d.max())

21.729357


In [100]:
import plotly.express as px
import plotly.offline as py


# fig, ax = plt.subplots(1, figsize=(16, 16))
result = pd.DataFrame({'pca_x': [x[0] for x in vectors_2d], 'pca_y': [x[1] for x in vectors_2d]})
# result = result[result['language'] == 'ru']
# result = result[result['pca_x'] < 3000000]
# result = result[result['pca_y'] < 1000]

fig = px.scatter(result, x='pca_x', y='pca_y')
fig.update_layout(width=int(1000), height=int(1000))
fig.show(renderer='browser')
# sns.scatterplot(x='pca_x', y='pca_y', hue='label', data=result, ax=ax, s=120)
# lim = (vectors_2d.min()-5, vectors_2d.max()+5)
# ax.set_xlim(3000000)
# ax.set_ylim(1000)
# ax.set_aspect('equal')
# ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)


In [49]:
vectors_3d = PCA(n_components=3, svd_solver='full', random_state=29).fit_transform(initial_vectors)

In [50]:
import plotly.express as px
import plotly.offline as py

result = pd.DataFrame({'pca_x': [x[0] for x in vectors_3d], 'pca_y': [x[1] for x in vectors_3d], 'pca_z': [x[2] for x in vectors_3d], 'language': df['language'], 'label': df['label']})
# result = result[result['language'] == 'ru']
# result = result[result['pca_x'] < 3000000]
# result = result[result['pca_y'] < 1000]

fig = px.scatter_3d(result, x='pca_x', y='pca_y', z='pca_z', color='label')
fig.update_layout(width=int(1000), height=int(1000))
fig.show(renderer='browser')